In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/NLP-2-main.zip"

extract_path = "/content/NLP-2-extracted"
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Files extracted to {extract_path}")

Files extracted to /content/NLP-2-extracted


In [4]:

print("Contents of the extracted folder:")
for item in os.listdir(extract_path):
    print(item)

Contents of the extracted folder:
NLP-2-main


In [5]:
!pip -q install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
# # Got a free API key from https://console.groq.com/keys
# import os
# from typing import Dict, List
# from groq import Groq

# os.environ["GROQ_API_KEY"] = "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"

# LLAMA3_70B_INSTRUCT = "llama-3.1-70b-versatile"
# DEFAULT_MODEL = LLAMA3_70B_INSTRUCT

# client = Groq()

# def chat_completion(
#     messages: List[Dict],
#     model = DEFAULT_MODEL,
#     temperature: float = 0.6,
#     top_p: float = 0.9,
# ) -> str:
#     response = client.chat.completions.create(
#         messages=messages,
#         model=model,
#         temperature=temperature,
#         top_p=top_p,
#     )
#     return response.choices[0].message.content

# def completion(
#     prompt: str,
#     model: str = DEFAULT_MODEL,
#     temperature: float = 0.6,
#     top_p: float = 0.9,
# ) -> str:
#     return chat_completion(
#         [{"role": "user", "content": prompt}],
#         model=model,
#         temperature=temperature,
#         top_p=top_p,
#     )

# def complete_and_print(prompt: str, model: str = DEFAULT_MODEL):
#     print(f'==============\n{prompt}\n==============')
#     response = completion(prompt, model)
#     print(response, end='\n\n')


In [ ]:
# complete_and_print("Hello, what model are you?")

In [13]:
import os
import time
from typing import Dict, List
from groq import Groq
from nltk.tokenize import sent_tokenize
import nltk

# Ensure NLTK data is downloaded
nltk.download('punkt')

# Function to get the GROQ API key
def get_groq_api_key():
    api_key = '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'
    api_key_2 ='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'
    api_key_3 = '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'
    if not api_key:
        api_key = input("Please enter your GROQ API Key: ")
        os.environ["GROQ_API_KEY"] = api_key_2
    return api_key

api_key = get_groq_api_key()
client = Groq()

LLAMA3_70B_INSTRUCT = "llama-3.1-70b-versatile"
DEFAULT_MODEL = LLAMA3_70B_INSTRUCT


# Function to estimate the number of tokens
def estimate_tokens(text):
    return int(len(text) / 4)


# Function to generate a chat completion using the Groq client
def chat_completion(
    messages: List[Dict],
    model=DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
    max_tokens: int = 256,
) -> str:
    global client  # Declare global client at the beginning
    try:
        response = client.chat.completions.create(
            messages=messages,
            model=model,
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred during API call: {e}")
        api_key = input("API key may be invalid or expired. Please enter a new GROQ API Key: ")
        os.environ["GROQ_API_KEY"] = api_key
        client = Groq()  # Reinitialize the client with the new API key
        # Retry the API call
        return chat_completion(
            messages,
            model=model,
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
        )

# Function to generate a completion with rate limit checks
def completion(
    prompt: str,
    model: str = DEFAULT_MODEL,
    temperature: float = 0.6,
    top_p: float = 0.9,
    max_tokens: int = 256,
) -> str:
    num_prompt_tokens = estimate_tokens(prompt)
    num_response_tokens = max_tokens
    total_tokens = num_prompt_tokens + num_response_tokens


    return chat_completion(
        [{"role": "user", "content": prompt}],
        model=model,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
    )

# Function to split text into chunks
def chunk_text(text, chunk_size=3000):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= chunk_size:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# Function to generate a Q&A pair from a text chunk
def generate_qa_pair(chunk):
    prompt = f"""Based on the following text, generate a single-line question and its answer. The question should be specific and answerable from the given text.

Example:
Text: The Eiffel Tower, located in Paris, France, was completed in 1889. It stands at a height of 324 meters and was the tallest man-made structure in the world for 41 years until the Chrysler Building in New York City was built in 1930.
Q: In what year was the Eiffel Tower completed?
A: The Eiffel Tower was completed in 1889.

Now, generate a question and answer for the following text:
{chunk}"""

    response = completion(prompt)
    qa_pair = response.strip().split('\n', 1)
    if len(qa_pair) == 2:
        return qa_pair[0].strip(), qa_pair[1].strip()
    else:
        return None, None

# Function to process documents in a folder and generate Q&A pairs
def process_documents(folder_path, output_file):
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for filename in os.listdir(folder_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(folder_path, filename)
                print(f"Processing: {filename}")
                with open(file_path, 'r', encoding='utf-8') as infile:
                    content = infile.read()
                chunks = chunk_text(content)
                total_chunks = len(chunks)
                chunks_processed = 0
                start_time = time.time()
                outfile.write(f"File: {filename}\n\n")
                for chunk in chunks:
                    question, answer = generate_qa_pair(chunk)
                    if question and answer:
                        outfile.write(f"Q: {question}\n")
                        outfile.write(f"A: {answer}\n\n")
                    chunks_processed += 1
                    elapsed_time = time.time() - start_time
                    avg_time_per_chunk = elapsed_time / chunks_processed
                    chunks_left = total_chunks - chunks_processed
                    eta = avg_time_per_chunk * chunks_left
                    print(f"Chunks processed: {chunks_processed}/{total_chunks}, ETA: {int(eta)} seconds")
                outfile.write("=" * 50 + "\n\n")
                print(f"Completed: {filename}")

# Example usage
folder_path = '/content/NLP-2-extracted/NLP-2-main/raw_data/General Info'  # Update with your folder path
output_file = 'generalinfo_output_qa.txt'      # Update with your desired output file

print('Process started')
process_documents(folder_path, output_file)
print(f"Q&A pairs have been generated and saved to {output_file}")
print('Process completed')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Process started
Processing: cmu_britannica.txt
Chunks processed: 1/1, ETA: 0 seconds
Completed: cmu_britannica.txt
Processing: pittsburgh_britannica.txt
An error occurred during API call: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-70b-versatile` in organization `org_01jamzny92ec9agske3vg20m4j` on : Limit 500000, Used 499483, Requested 893. Please try again in 1m4.8898s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
API key may be invalid or expired. Please enter a new GROQ API Key: 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
Chunks processed: 1/3, ETA: 31 seconds
Chunks processed: 2/3, ETA: 8 seconds
Chunks processed: 3/3, ETA: 0 seconds
Completed: pittsburgh_britannica.txt
Processing: pittsburgh_pa.txt
Chunks processed: 1/105, ETA: 45 seconds
Chunks processed: 2/105, ETA: 45 seconds
Chunks processed: 3/105, ETA: 144 seconds
Chunks processed: 4/105, ETA: 174 seconds


In [ ]:
folder_path = '/content/NLP-2-extracted/NLP-2-main/raw_data/Sports'  # Update with your folder path
output_file = 'Sports_output_qa.txt'      # Update with your desired output file

print('Process started')
process_documents(folder_path, output_file)
print(f"Q&A pairs have been generated and saved to {output_file}")
print('Process completed')

Process started
Processing: steelers.txt
Chunks processed: 1/435, ETA: 436 seconds
Chunks processed: 2/435, ETA: 321 seconds
Chunks processed: 3/435, ETA: 287 seconds
Chunks processed: 4/435, ETA: 263 seconds
Chunks processed: 5/435, ETA: 348 seconds
Chunks processed: 6/435, ETA: 326 seconds
Chunks processed: 7/435, ETA: 305 seconds
Chunks processed: 8/435, ETA: 286 seconds
Chunks processed: 9/435, ETA: 273 seconds
Chunks processed: 10/435, ETA: 263 seconds
Chunks processed: 11/435, ETA: 253 seconds
Chunks processed: 12/435, ETA: 245 seconds
Chunks processed: 13/435, ETA: 254 seconds
Chunks processed: 14/435, ETA: 249 seconds
Chunks processed: 15/435, ETA: 243 seconds
Chunks processed: 16/435, ETA: 457 seconds
Chunks processed: 17/435, ETA: 486 seconds
Chunks processed: 18/435, ETA: 485 seconds
Chunks processed: 19/435, ETA: 484 seconds
Chunks processed: 20/435, ETA: 467 seconds
Chunks processed: 21/435, ETA: 457 seconds
Chunks processed: 22/435, ETA: 451 seconds
Chunks processed: 23/4